In [ ]:
from dotenv import load_dotenv
import geopandas as gpd                     # type: ignore
import pandas as pd
import os, sys

sys.path.append('../')
from Shared.DataService import DataService  # type: ignore

In [ ]:
SOIL_GEOM_TABLE = 'soil_geometry'
SOIL_COMP_TABLE = 'soil_components'
SOIL_SURRONDINGS_TABLE = 'soil_surronding_land'
SOIL_DATA_TABLE = 'soil_data'

load_dotenv()
PG_DB = os.getenv('POSTGRES_DB')
PG_ADDR = os.getenv('POSTGRES_ADDR')
PG_PORT = os.getenv('POSTGRES_PORT')
PG_USER = os.getenv('POSTGRES_USER')
PG_PW = os.getenv('POSTGRES_PW')

In [ ]:
db = DataService(PG_DB, PG_ADDR, PG_PORT, PG_USER, PG_PW)
conn = db.connect()

In [ ]:
# load soil geometry data
file = 'data/ca_all_slc_v3r2.shp'
soilGeom = gpd.read_file(file, encoding='utf-8')
soilGeom = soilGeom.set_crs(crs='EPSG:4326', allow_override=True)
soilGeom = soilGeom.to_crs(crs='EPSG:3347')

soilGeom.drop(columns=['ECO_ID'], inplace=True)

soilGeom.rename(columns={soilGeom.columns[0]: 'area'}, inplace=True)
soilGeom.rename(columns={soilGeom.columns[1]: 'perimeter'}, inplace=True)
soilGeom.rename(columns={soilGeom.columns[2]: 'poly_id'}, inplace=True)

soilGeom.to_postgis(SOIL_GEOM_TABLE, conn, index=False, if_exists='replace')

In [ ]:
# load soil component data
file = 'data/ca_all_slc_v3r2_cmp.dbf'
soilComp = gpd.read_file(file, encoding='utf-8')
soilComp = pd.DataFrame(soilComp.drop(columns='geometry'))

# load soil component rating data
file = 'data/ca_all_slc_v3r2_crt.dbf'
soilCompRating = gpd.read_file(file, encoding='utf-8')
soilCompRating = pd.DataFrame(soilCompRating.drop(columns='geometry'))

# merge both the soil component and soil component rating data together
soilComp = soilComp.merge(soilCompRating, on='CMP_ID')

soilComp.drop(columns=['CMP_ID', 'RESTR_TYPE'], inplace=True)

soilComp.rename(columns={soilComp.columns[0]: 'poly_id'}, inplace=True)
soilComp.rename(columns={soilComp.columns[1]: 'cmp'}, inplace=True)
soilComp.rename(columns={soilComp.columns[2]: 'percent'}, inplace=True)
soilComp.rename(columns={soilComp.columns[3]: 'slope'}, inplace=True)
soilComp.rename(columns={soilComp.columns[4]: 'stone'}, inplace=True)
soilComp.rename(columns={soilComp.columns[5]: 'surface_area'}, inplace=True)
soilComp.rename(columns={soilComp.columns[6]: 'province'}, inplace=True)
soilComp.rename(columns={soilComp.columns[7]: 'soil_code'}, inplace=True)
soilComp.rename(columns={soilComp.columns[8]: 'modifier'}, inplace=True)
soilComp.rename(columns={soilComp.columns[9]: 'profile'}, inplace=True)
soilComp.rename(columns={soilComp.columns[10]: 'soil_id'}, inplace=True)
soilComp.rename(columns={soilComp.columns[11]: 'coarse_frag_1'}, inplace=True)
soilComp.rename(columns={soilComp.columns[12]: 'coarse_frag_2'}, inplace=True)
soilComp.rename(columns={soilComp.columns[13]: 'depth'}, inplace=True)
soilComp.rename(columns={soilComp.columns[14]: 'water_holding_cap'}, inplace=True)

soilComp.to_sql(SOIL_COMP_TABLE, conn, index=False, if_exists='replace')

In [ ]:
# load surronding landscape data
file = 'data/ca_all_slc_v3r2_lat.dbf'
soilSurrondings = gpd.read_file(file, encoding='utf-8')
soilSurrondings = pd.DataFrame(soilSurrondings.drop(columns='geometry'))

soilSurrondings.rename(columns={soilSurrondings.columns[0]: 'poly_id'}, inplace=True)
soilSurrondings.rename(columns={soilSurrondings.columns[1]: 'land_area'}, inplace=True)
soilSurrondings.rename(columns={soilSurrondings.columns[2]: 'water_area'}, inplace=True)
soilSurrondings.rename(columns={soilSurrondings.columns[3]: 'fresh_area'}, inplace=True)
soilSurrondings.rename(columns={soilSurrondings.columns[4]: 'ocean_area'}, inplace=True)
soilSurrondings.rename(columns={soilSurrondings.columns[5]: 'total_area'}, inplace=True)

soilSurrondings.to_sql(SOIL_SURRONDINGS_TABLE, conn, index=False, if_exists='replace')

In [ ]:
# load the data for the soils present in Manitoba
file = 'data/soil_name_mb_v2r20130705.dbf'
mbSoilName = gpd.read_file(file, encoding='utf-8')
mbSoilName = pd.DataFrame(mbSoilName.drop(columns='geometry'))

file = 'data/soil_layer_mb_v2r20130705.dbf'
mbSoilLayer = gpd.read_file(file, encoding='utf-8')
mbSoilLayer = pd.DataFrame(mbSoilLayer.drop(columns='geometry'))

mbSoil = mbSoilName.merge(mbSoilLayer)


# load the data for the soils present in Alberta
file = 'data/soil_name_ab_v2r20140529.dbf'
abSoilName = gpd.read_file(file, encoding='utf-8')
abSoilName = pd.DataFrame(abSoilName.drop(columns='geometry'))

file = 'data/soil_layer_ab_v2r20140529.dbf'
abSoilLayer = gpd.read_file(file, encoding='utf-8')
abSoilLayer = pd.DataFrame(abSoilLayer.drop(columns='geometry'))

abSoil = abSoilName.merge(abSoilLayer)


# load the data for the soils present in Saskatchewan
file = 'data/soil_name_sk_v2r20130705.dbf'
skSoilName = gpd.read_file(file, encoding='utf-8')
skSoilName = pd.DataFrame(skSoilName.drop(columns='geometry'))

file = 'data/soil_layer_sk_v2r20130705.dbf'
skSoilLayer = gpd.read_file(file, encoding='utf-8')
skSoilLayer = pd.DataFrame(skSoilLayer.drop(columns='geometry'))

skSoil = skSoilName.merge(skSoilLayer)


soilData = pd.concat([mbSoil, abSoil, skSoil])

soilData.drop(columns=['PROFILE', 'ORDER2', 'G_GROUP2', 'S_GROUP2', 'ORDER3', 'G_GROUP3', 'S_GROUP3'], inplace=True)

soilData.rename(columns={soilData.columns[0]: 'id'}, inplace=True)
soilData.rename(columns={soilData.columns[1]: 'province'}, inplace=True)
soilData.rename(columns={soilData.columns[2]: 'code'}, inplace=True)
soilData.rename(columns={soilData.columns[3]: 'modifier'}, inplace=True)
soilData.rename(columns={soilData.columns[4]: 'name'}, inplace=True)
soilData.rename(columns={soilData.columns[5]: 'kind'}, inplace=True)
soilData.rename(columns={soilData.columns[6]: 'water_table'}, inplace=True)
soilData.rename(columns={soilData.columns[7]: 'root_restrict'}, inplace=True)
soilData.rename(columns={soilData.columns[8]: 'restr_type'}, inplace=True)
soilData.rename(columns={soilData.columns[9]: 'drainage'}, inplace=True)
soilData.rename(columns={soilData.columns[10]: 'parent_material_texture_1'}, inplace=True)
soilData.rename(columns={soilData.columns[11]: 'parent_material_texture_2'}, inplace=True)
soilData.rename(columns={soilData.columns[12]: 'parent_material_texture_3'}, inplace=True)
soilData.rename(columns={soilData.columns[13]: 'parent_material_chemical_1'}, inplace=True)
soilData.rename(columns={soilData.columns[14]: 'parent_material_chemical_2'}, inplace=True)
soilData.rename(columns={soilData.columns[15]: 'parent_material_chemical_3'}, inplace=True)
soilData.rename(columns={soilData.columns[16]: 'mode_of_depo_1'}, inplace=True)
soilData.rename(columns={soilData.columns[17]: 'mode_of_depo_2'}, inplace=True)
soilData.rename(columns={soilData.columns[18]: 'mode_of_depo_3'}, inplace=True)
soilData.rename(columns={soilData.columns[19]: 'layer_no'}, inplace=True)
soilData.rename(columns={soilData.columns[20]: 'u_depth'}, inplace=True)
soilData.rename(columns={soilData.columns[21]: 'l_depth'}, inplace=True)
soilData.rename(columns={soilData.columns[22]: 'hzn_lit'}, inplace=True)
soilData.rename(columns={soilData.columns[23]: 'hzn_mas'}, inplace=True)
soilData.rename(columns={soilData.columns[24]: 'hzn_suf'}, inplace=True)
soilData.rename(columns={soilData.columns[25]: 'hzn_mod'}, inplace=True)
soilData.rename(columns={soilData.columns[26]: 'percnt_coarse_frag'}, inplace=True)
soilData.rename(columns={soilData.columns[27]: 'sand_texture'}, inplace=True)
soilData.rename(columns={soilData.columns[28]: 'percnt_v_fine_sand'}, inplace=True)
soilData.rename(columns={soilData.columns[29]: 'total_sand'}, inplace=True)
soilData.rename(columns={soilData.columns[30]: 'total_silt'}, inplace=True)
soilData.rename(columns={soilData.columns[31]: 'total_clay'}, inplace=True)
soilData.rename(columns={soilData.columns[32]: 'percnt_carbon'}, inplace=True)
soilData.rename(columns={soilData.columns[33]: 'calcium_ph'}, inplace=True)
soilData.rename(columns={soilData.columns[34]: 'proj_ph'}, inplace=True)
soilData.rename(columns={soilData.columns[35]: 'percnt_base_sat'}, inplace=True)
soilData.rename(columns={soilData.columns[36]: 'cec'}, inplace=True)
soilData.rename(columns={soilData.columns[37]: 'ksat'}, inplace=True)
soilData.rename(columns={soilData.columns[38]: 'water_reten_0'}, inplace=True)
soilData.rename(columns={soilData.columns[39]: 'water_reten_10'}, inplace=True)
soilData.rename(columns={soilData.columns[40]: 'water_reten_33'}, inplace=True)
soilData.rename(columns={soilData.columns[41]: 'water_reten_1500'}, inplace=True)
soilData.rename(columns={soilData.columns[42]: 'bulk_density'}, inplace=True)
soilData.rename(columns={soilData.columns[43]: 'elec_cond'}, inplace=True)
soilData.rename(columns={soilData.columns[44]: 'calc_equiv'}, inplace=True)
soilData.rename(columns={soilData.columns[45]: 'decomp_class'}, inplace=True)
soilData.rename(columns={soilData.columns[46]: 'percnt_wood'}, inplace=True)

soilData.to_sql(SOIL_DATA_TABLE, conn, schema='public', if_exists='append', index=False)

In [ ]:
db.cleanup()